In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn import linear_model
import statsmodels.formula.api as smf
import statsmodels.api as sm
from scipy.stats import bartlett, levene, jarque_bera, normaltest
from statsmodels.tsa.stattools import acf
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'

In [2]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

weather_df = pd.read_sql_query('select * from weatherinszeged', con=engine)

engine.dispose()

In [3]:
weather_df.head()

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.


In [4]:
#removing rows that have 0 humidity as this is physically impossible
#outside of a laboratory environment
weather_df['temp_diff'] = weather_df['temperature'] - weather_df['apparenttemperature']
no_humidity = weather_df.loc[weather_df['humidity'] == 0]

weather_df.drop(no_humidity.index, inplace=True)
len(weather_df)

96431

In [5]:
#1st iteration
Y = weather_df['temp_diff']
X = weather_df[[
    'humidity', 
    'windspeed'
]]

X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              temp_diff   R-squared:                       0.290
Model:                            OLS   Adj. R-squared:                  0.290
Method:                 Least Squares   F-statistic:                 1.965e+04
Date:                Sun, 23 Jun 2019   Prob (F-statistic):               0.00
Time:                        05:58:11   Log-Likelihood:            -1.7023e+05
No. Observations:               96431   AIC:                         3.405e+05
Df Residuals:                   96428   BIC:                         3.405e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.4585      0.021   -116.947      0.000      -2.500      -2.417
humidity       3.0537      0.024    127.531      0.000       3.007       3.101
windspeed      0.1194      0.001    176.627      0.000       0.118       0.121
==============================================================================
Omnibus:                     3512.457   Durbin-Watson:                   0.267
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3994.499
Skew:                           0.459   Prob(JB):                         0.00
Kurtosis:                       3.387   Cond. No.                         88.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Based on the R-squared values, the model above is pretty weak. Both the adjusted and non-adjusted R-squared values are 0.290. These two features alone do not satisfactorily explain the temperature differences.

In [6]:
weather_df['humid_windspeed'] = weather_df['humidity'] * weather_df['windspeed']

In [7]:
#2nd iteration
Y = weather_df['temp_diff']
X = weather_df[[
    'humidity', 
    'windspeed',
    'humid_windspeed'
]]

X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              temp_diff   R-squared:                       0.344
Model:                            OLS   Adj. R-squared:                  0.344
Method:                 Least Squares   F-statistic:                 1.682e+04
Date:                Sun, 23 Jun 2019   Prob (F-statistic):               0.00
Time:                        05:58:11   Log-Likelihood:            -1.6641e+05
No. Observations:               96431   AIC:                         3.328e+05
Df Residuals:                   96427   BIC:                         3.329e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.0872      0.033     -2.611      0.009      -0.153      -0.022
humidity           -0.1752      0.043     -4.079      0.000      -0.259      -0.091
windspeed          -0.0918      0.002    -37.331      0.000      -0.097      -0.087
humid_windspeed     0.2990      0.003     89.078      0.000       0.292       0.306
==============================================================================
Omnibus:                     4017.847   Durbin-Watson:                   0.265
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7049.641
Skew:                           0.346   Prob(JB):                         0.00
Kurtosis:                       4.130   Cond. No.                         194.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The R-squared values have gone up in this iteration of the model. Both are 0.344. Based on that the model has improved.

In [8]:
#3rd iteration
Y = weather_df['temp_diff']
X = weather_df[[
    'humidity', 
    'windspeed',
    'visibility'
]]

X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              temp_diff   R-squared:                       0.305
Model:                            OLS   Adj. R-squared:                  0.305
Method:                 Least Squares   F-statistic:                 1.410e+04
Date:                Sun, 23 Jun 2019   Prob (F-statistic):               0.00
Time:                        05:58:11   Log-Likelihood:            -1.6917e+05
No. Observations:               96431   AIC:                         3.384e+05
Df Residuals:                   96427   BIC:                         3.384e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.6034      0.028    -57.566      0.000      -1.658      -1.549
humidity       2.6334      0.025    103.778      0.000       2.584       2.683
windspeed      0.1200      0.001    179.405      0.000       0.119       0.121
visibility    -0.0534      0.001    -46.146      0.000      -0.056      -0.051
==============================================================================
Omnibus:                     3469.598   Durbin-Watson:                   0.282
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4025.476
Skew:                           0.444   Prob(JB):                         0.00
Kurtosis:                       3.464   Cond. No.                         131.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Re-iterating the first version of the model with the visibility feature added has improved the R-squared values to 0.305, but they are not as high as the second iteration. Based on the higher R-squared value and the larger F-statistic, I think the humid_windspeed feature is more useful in the model than visibility.

The second iteration of the model seems to be the best based on the AIC and BIC scores. They are both slightly lower than the 3rd iteration and significantly lower than the 1st iteration.